# Задание проекта: визуализировать данные Тинькофф квеста в динамике по неделям и месяцам

In [ ]:
import psycopg2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from plotly import graph_objects as go

pd.set_option('display.max_columns', 25)

In [ ]:
conn = psycopg2.connect(
    host="158.160.52.106",
    port=5432,
    database="postgres",
    user="student",
    password="JvLda93aA",
)


In [ ]:
df = pd.read_sql("""select
	*
	,extract (day from registration_week - visit_week) / 7 as registration_visit
	,extract (day from application_week - visit_week) / 7 as application_visit
	,extract (day from game_week - visit_week) / 7 as game_visit
from (
	select 
		c.client_rk
		,c.visit_dttm 
		,date_trunc('month', c.visit_dttm) as visit_month
		,date_trunc('week', c.visit_dttm) as visit_week
		,a2.account_rk 
		,a2.registration_dttm
		,date_trunc('month', a2.registration_dttm) as registration_month
		,date_trunc('week', a2.registration_dttm) as registration_week
		,a.game_rk 
		,a.application_dttm
		,date_trunc('month', a.application_dttm) as application_month
		,date_trunc('week', a.application_dttm) as application_week
		,g.game_flg
		,g.game_dttm
		,date_trunc('month', g.game_dttm) as game_month
		,date_trunc('week', g.game_dttm) as game_week 
	from msu_analytics.game g
	full join msu_analytics.application a 
	on g.game_rk = a.game_rk
	full join msu_analytics.account a2
	on a.account_rk = a2.account_rk
	right join msu_analytics.client c 
	on a2.client_rk = c.client_rk
) t1;""", conn)

In [ ]:
visit_week_df = pd.DataFrame(df.drop_duplicates(subset='client_rk')['visit_week'].value_counts()) \
    .reset_index(level=0) \
    .rename(columns={'index': 'week', 'visit_week': 'visit_count'}) \
    .sort_values(by='week')

registration_week_df = pd.DataFrame(df.drop_duplicates(subset=['account_rk'])['registration_week'].value_counts()) \
    .reset_index(level=0) \
    .rename(columns={'index': 'week', 'registration_week': 'registration_count'}) \
    .sort_values(by='week')

application_week_df = pd.DataFrame(df['application_week'].value_counts()) \
    .reset_index(level=0) \
    .rename(columns={'index': 'week', 'application_week': 'application_count'}) \
    .sort_values(by='week')

game_week_df = pd.DataFrame(df.query('game_flg == 1')['game_week'].value_counts()) \
    .reset_index(level=0) \
    .rename(columns={'index': 'week', 'game_week': 'game_count'}) \
    .sort_values(by='week')

# Объединение датафрэймов в один (для удобности)
full_df_weeks = pd.merge(
                    pd.merge(
                        pd.merge(visit_week_df,registration_week_df, how='outer') \
                    , application_week_df, how='outer') \
                , game_week_df, how='outer')

for column in full_df_weeks.columns:
    if column != 'week':
        full_df_weeks[f'{column}_percent'] = round(full_df_weeks[column] * 100 / full_df_weeks['visit_count'], 1)

# Чтобы на графике значения на каждый из этапов были между понедельниками добавим промежуточную точку between_week
full_df_weeks['between_week'] = full_df_weeks['week'] + pd.to_timedelta(3.5, unit="D")
full_df_weeks

In [ ]:
client_num = df['client_rk'].nunique()
account_num = df['account_rk'].nunique()
application_num = df.drop_duplicates(subset='account_rk')['game_rk'].count()
game_num = df['game_flg'].sum()

print(client_num,account_num,application_num, game_num)

In [ ]:
fig = go.Figure()

# Настройка графика
fig.update_layout(autosize=False,
    width=800,
    height=800,
    title = 'Воронка по неделям',
    title_font_size=20)

# Построение графика
for ind in full_df_weeks.index:
    fig.add_trace(go.Funnel(
        name = full_df_weeks['week'][ind].strftime('%Y-%m-%d'),
        y = ['Зашли на сайт',
             'Зарегистрировались',
             'Оставили заявку на игру',
             'Пришли на игру'],
        x = [full_df_weeks['visit_count'][ind], 
             full_df_weeks['registration_count'][ind], 
             full_df_weeks['application_count'][ind], 
             full_df_weeks['game_count'][ind]],
        textposition = "inside",
        textinfo = "value+percent initial"))
    


fig.show()

In [ ]:
# Построение графика
plt.figure(2, figsize=(15,10), dpi=80)

plt.plot(full_df_months['between_month'], full_df_months['visit_count'], '.-')


plt.plot(full_df_months['between_month'], full_df_months['registration_count'], '.-')


plt.plot(full_df_months['between_month'], full_df_months['application_count'], '.-')


plt.plot(full_df_months['between_month'], full_df_months['game_count'], '.-')


# Добавление аннотаций
for ind in full_df_months.index:
    plt.annotate(str(full_df_months['visit_count_percent'][ind]) + '%', 
                 (full_df_months['between_month'][ind], full_df_months['visit_count'][ind]), 
                 textcoords="offset points",
                 xytext=(0,10),
                 ha='center')

for ind in full_df_months.index:
    plt.annotate(str(full_df_months['registration_count_percent'][ind]) + '%', 
                 (full_df_months['between_month'][ind], full_df_months['registration_count'][ind]),
                 textcoords="offset points",
                 xytext=(0,10),
                 ha='center')
    
for ind in full_df_months.index:
    plt.annotate(str(full_df_months['application_count_percent'][ind]) + '%', 
                 (full_df_months['between_month'][ind], full_df_months['application_count'][ind]),
                 textcoords="offset points",
                 xytext=(0,10),
                 ha='center')
    
for ind in full_df_months.index:
    plt.annotate(str(full_df_months['game_count_percent'][ind]) + '%', 
                 (full_df_months['between_month'][ind], full_df_months['game_count'][ind]),
                 textcoords="offset points",
                 xytext=(0,-10),
                 ha='center')

# Настройка графика
plt.title('Воронка по месяцам №1', fontsize=20)
plt.gca().set(ylabel='Количество пользователей', xlabel='Месяцы')
plt.grid(linestyle='--', alpha=0.5)
plt.yticks(np.arange(0, max(full_df_months['visit_count'])+ 10, 50))
plt.xticks(pd.date_range(min(full_df_months['month']) , max(full_df_months['month']), periods=full_df_months['month'].count()).tolist())
plt.gca().legend(('Зашли на сайт', 'Зарегистрировались', 'Оставили заявку', 'Пришли на игру'))

plt.show()

In [ ]:
# Настройка воронки
fig = go.Figure()

fig.update_layout(autosize=False,
    width=800,
    height=800,
    title = 'Воронка по месяцам №2',
    title_font_size=20)

# Построение воронки
for ind in full_df_months.index:
    fig.add_trace(go.Funnel(
        name = full_df_months['month'][ind].strftime('%Y-%m'),
        y = ['Зашли на сайт',
             'Зарегистрировались',
             'Оставили заявку на игру',
             'Пришли на игру'],
        x = [full_df_months['visit_count'][ind], 
              full_df_months['registration_count'][ind], 
              full_df_months['application_count'][ind], 
              full_df_months['game_count'][ind]],
        textposition = "inside",
        textinfo = "value+percent initial"))
    


fig.show()